# Maximizing Revenue for Taxi Cab Drivers Through Payment Type Analysis

## Problem Statement
In the fast-paced taxi booking sector, making the most of revenue is essential for long-term success and driver happiness. Our goal is to use data-driven insights to maximise revenue streams for taxi drivers in order to meet this need. Our research aims to determine whether payment methods have an impact on fare pricing by focusing on the relationship between payment type and fare amount.


## Objective
This project's main goal is to run an A/B test to examine the relationship between the total fare and the method of payment. We use Python hypothesis testing and descriptive statistics to extract useful information that can help taxi drivers generate more cash. In particular, we want to find out if there is a big difference in the fares for those who pay with credit cards versus those who pay with cash.


## Research Question
Is there a relationship between total fare amount and payment type and can we nudge customers towards payment methods that generate higher revenue for drivers, without negatively impacting customer experience?

# Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')

In [23]:
df = pq.read_table(r"D:\UNH Materials\Projects\Maximizing-revenue\data\yellow_tripdata_2020-01.parquet")
df= df.to_pandas()

In [24]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,None
1,1,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5,None
2,1,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5,None
3,1,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0,None
4,2,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0,None


# Exploratory Data Analysis

In [27]:
df.shape

(6405008, 19)

In [28]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                      object
dtype: object

In [29]:
# Calculate duration

df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].dt.total_seconds()/60

In [30]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,None,4.800000
1,1,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5,None,7.416667
2,1,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5,None,6.183333
3,1,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0,None,4.850000
4,2,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0,None,2.300000


In [31]:
df = df[['passenger_count', 'payment_type', 'total_amount', 'trip_distance', 'duration']]

In [32]:
# Check missing values
df.isnull().sum()

passenger_count    65441
payment_type           0
total_amount           0
trip_distance          0
duration               0
dtype: int64

In [33]:
df.shape

(6405008, 5)

In [34]:
# Percentage of missing values

(df.isnull().sum()[0] / df.shape[0]) * 100

1.021716132126611

Just 1% of missing values, easily dropable

In [35]:
df.dropna(inplace=True)

In [36]:
df.shape

(6339567, 5)

In [37]:
# Change data dtype of passenger count column

df['passenger_count'] = df['passenger_count'].astype('int64')
df['payment_type'] = df['payment_type'].astype('int64')

In [38]:
df.head()

,passenger_count,payment_type,total_amount,trip_distance,duration
0,1,1,11.27,1.2,4.800000
1,1,1,12.30,1.2,7.416667
2,1,1,10.80,0.6,6.183333
3,1,1,8.16,0.8,4.850000
4,1,2,4.80,0.0,2.300000


In [39]:
# Check for duplicate values

df[df.duplicated()]

,passenger_count,payment_type,total_amount,trip_distance,duration
2056,1,2,10.80,0.00,0.000000
3729,1,2,11.30,1.50,7.166667
5172,1,2,7.80,0.20,3.666667
5735,1,2,9.30,0.80,5.116667
7483,1,1,12.35,1.20,6.550000
...,...,...,...,...,...
6339557,1,1,10.56,0.74,4.183333
6339558,1,2,11.80,1.63,8.800000
6339559,1,1,14.76,1.81,8.016667
6339562,1,1,17.76,2.10,14.233333


In [40]:
df[df.duplicated()].shape[0] / df.shape[0] * 100

28.189149195836244

28% data is duplicated/reduntant 

In [41]:
df.drop_duplicates(inplace = True)

In [42]:
df.shape

(4552497, 5)

In [44]:
# Distribution of passenger_counts

df['passenger_count'].value_counts(normalize=True)

1    0.648233
2    0.175584
3    0.052069
5    0.047670
6    0.028335
4    0.026260
0    0.021835
7    0.000006
9    0.000004
8    0.000004
Name: passenger_count, dtype: float64

In [45]:
df['payment_type'].value_counts(normalize=True)

1    7.511291e-01
2    2.387808e-01
3    6.318950e-03
4    3.770898e-03
5    2.196597e-07
Name: payment_type, dtype: float64

We only need payment type 1 and 2, so we will filter the other three payment types

In [48]:
df = df[df['payment_type'] < 3]
df = df[(df['passenger_count'] > 0) & (df['passenger_count'] < 7)]

In [49]:
df.shape

(4409464, 5)

In [51]:
# Change variable name

df['payment_type'].replace([1, 2], ['Card', 'Cash'], inplace=True)

In [52]:
df.head()

,passenger_count,payment_type,total_amount,trip_distance,duration
0,1,Card,11.27,1.2,4.800000
1,1,Card,12.30,1.2,7.416667
2,1,Card,10.80,0.6,6.183333
3,1,Card,8.16,0.8,4.850000
4,1,Cash,4.80,0.0,2.300000
